Подключаем необходимые библиотеки

In [3]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_iris


In [4]:
iris_dataset = load_iris()

df = pd.DataFrame(data=iris_dataset.data, columns=iris_dataset.feature_names)
df['target'] = iris_dataset.target
df['target'] = df.target.apply(lambda v: iris_dataset.target_names[v])

print(len(df))

df.head()

150


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


Создадим функцию, вычисляющую энтропию

In [5]:

from math import log2

def entropy(targets: pd.Series) -> float:
    class_probas = targets.value_counts(normalize=True)
    return -class_probas.apply(lambda p: p * log2(p)).sum()
eps = 1e-7


Проверка условий на истинность

In [6]:
assert abs(entropy(pd.Series(['a', 'a', 'a'])) - 0.) < eps
assert abs(entropy(pd.Series(['a', 'a', 'b'])) - 0.9182958340) < eps
assert abs(entropy(pd.Series(['a', 'a', 'b', 'b', 'c', 'a', 'a', 'b'])) - 1.4056390622) < eps
assert abs(entropy(pd.Series(['a', 'b', 'c', 'd'])) - 2) < eps


Напишем функцию, которая применяется для оценки качества разбиения при решении задач классификации. Функция будет вычислять значение энтропии после разделения выборкина две части

In [7]:
def information_gain(before_split: pd.Series, split_left: pd.Series, split_right: pd.Series) -> float:
    e_before_split = entropy(before_split)
    e_left, left_proportion = entropy(split_left), len(split_left) / len(before_split)
    e_right, right_proportion = entropy(split_right), len(split_right) / len(before_split)
    return (e_before_split - left_proportion * e_left - right_proportion * e_right)


In [8]:
assert abs(information_gain(pd.Series(['a', 'a', 'b', 'b']), pd.Series(['a', 'a']), pd.Series(['b', 'b'])) - 1.0) < eps
assert abs(information_gain(pd.Series(['a', 'b', 'c', 'b']), pd.Series(['a', 'c']), pd.Series(['b', 'b'])) - 1.0) < eps
assert abs(information_gain(pd.Series(['a', 'b', 'c', 'd']), pd.Series(['a', 'b']), pd.Series(['c', 'd'])) - 1.0) < eps
assert abs(information_gain(pd.Series(['a', 'a', 'c', 'd']), pd.Series(['a', 'c']), pd.Series(['a', 'd'])) - 0.5) < eps


Теперь напишем функцию, которая разбивает переданные данные на части по указанному количеству шагов, обучает модель на каждой части и возвращает среднюю метрику качества модели и название используемого классификатора

In [9]:
from typing import Tuple


def split_by_feature(x: pd.DataFrame, y: pd.Series, feature: str, steps: int = 100) -> Tuple[float, float]:
    min_value, max_value = x[feature].min(), x[feature].max()
    thresholds = np.linspace(min_value, max_value, steps)
    best_gain, best_threshold = 0, None
    x_parts = np.array_split(x[feature], steps)
    y_parts = np.array_split(y, steps)
    for threshold in thresholds:
        ### Ваш код
        info_gains = []
        for i in range(steps):
            y_true = y_parts[i]
            split_left = y_true[x_parts[i] <= threshold]
            split_right = y_true[x_parts[i] > threshold]
            info_gains.append(information_gain(y_true, split_left, split_right))
        mean_info_gain = sum(info_gains) / len(info_gains)
        if mean_info_gain > best_gain:
            bast_gain = mean_info_gain
            best_threshold = threshold

    return best_gain, best_threshold


В описсанной ниже функции разбиения возвращается наилучший информационный выигрыш, соответствующий столбцу, индексу столбца и пороговому значению

In [10]:

def split_by_features(x: pd.DataFrame, y: pd.Series, steps: int = 100) -> Tuple[float, str]:
    best_gain, best_threshold, best_feature, best_idx = 0, None, None, None

    for idx, feature in enumerate(x.columns):
        ### Ваш код
        thresholds = np.linspace(x[feature].min(), x[feature].max(), steps)
        for threshold in thresholds:
            split_left = y[x[feature] <= threshold]
            split_right = y[x[feature] > threshold]
            gain = information_gain(y, split_left, split_right)
            if gain > best_gain:
                best_gain = gain
                best_feature = feature
                best_idx = idx
                best_threshold = threshold

    return best_gain, best_feature, best_idx, best_threshold

Опишем вершину принятия решений. В вершине мы либо храним правило, либо доминантный класс.

In [32]:

from dataclasses import dataclass
from typing import Any, Optional

@dataclass
class DecisionNode:
    feature: Optional[str] = None
    feature_idx: Optional[int] = None
    threshold: Optional[float] = None
    dominative_class: Any = None

    @classmethod
    def make_leaf(cls, values: pd.Series) -> 'DecisionNode':
        class_counts = values.value_counts().to_dict()
        dominative_class = max(class_counts, key=lambda c: class_counts[c])
        return cls(dominative_class=dominative_class)

    @classmethod
    def make_node(
        cls, best_feature: str, best_feature_idx: int, best_threshold: float
    ) -> 'DecisionNode':
        return cls(
            feature=best_feature,
            feature_idx=best_feature_idx,
            threshold=best_threshold
        )

    @property
    def is_leaf(self) -> bool:
        return bool(self.dominative_class)



Создадим класс. Метод fit будет отвечать за тренировку классификатора. Он перебирает все признаки и все уникальные значения каждого признака, строит решающее правило на основе признака и порога, вычисляет ошибку классификации для каждого правила и выбирает правило с минимальной ошибкой.


Метод predict использует сохраненный ранее узел дерева, который содержит информацию о признаке, пороге, левом и правом поддеревьях, исходных данных и метках листьев. Затем он рекурсивно проходит по дереву, начиная с корневого узла, и классифицирует новые данные, используя правила, сохраненные в каждом узле.

In [29]:
class DecisionStump:
    def __init__(self):
        # Будем хранить внутри пня информацию о лучшем признаке и лучшей границе этого признака
        self._node: Optional[DecisionNode] = None
        self._left_node: Optional[DecisionNode] = None
        self._right_node: Optional[DecisionNode] = None
        self.feature_index = None
    
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series) -> None:
        # _, feature, feature_idx, threshold = split_by_features(X_train, y_train)
        # x_left, x_right = X_train[X_train[feature] < threshold], X_train[X_train[feature] >= threshold]
        # y_left, y_right = y_train[x_left.index], y_train[x_right.index]
        # ### Ваш код
        # self.feature = None  # инициализируем признак, на котором делается разбиение
        # self.threshold = None  # инициализируем порог разбиения
        # self.alpha = None  # инициализируем вес классификатора
        # best_error = float("inf")  # инициализируем минимальную ошибку классификации

        # for feature in X_train.columns:  # перебираем все признаки
        #     for threshold in X_train[feature].unique():  # перебираем все уникальные значения признака
        #         # строим правило на основе признака и порога
        #         y_pred = np.where(X_train[feature] <= threshold, -1, 1)
        #         # вычисляем ошибку классификации
        #         error = np.sum(y_pred != y_train)
        #         if error < best_error:  # если ошибка меньше текущей минимальной
        #             best_error = error  # обновляем минимальную ошибку
        #             self.feature = feature  # сохраняем признак и порог
        #             self.threshold = threshold

        # # вычисляем вес классификатора

        # self.alpha = 0.5 * np.log((1 - best_error) / best_error)
        _, feature, feature_idx, threshold = split_by_features(X_train, y_train)
        x_left, x_right = X_train[X_train[feature] < threshold], X_train[X_train[feature] >= threshold]
        y_left, y_right = y_train[x_left.index], y_train[x_right.index]
        self._node = DecisionNode.make_node(feature, feature_idx, threshold)
        self._left_node = DecisionNode.make_leaf(y_left)
        self._right_node = DecisionNode.make_leaf(y_right)

        

    def predict(self, X: pd.DataFrame) -> pd.Series:
        # x = X.copy()
        # node = self._node
        # left_samples = x[x[node.feature] < node.threshold]
        # right_samples = x[x[node.feature] >= node.threshold]
        # ### Ваш код
        # predictions = pd.Series(np.zeros(len(x)), index=x.index)

        # while node:
        #     if node.is_leaf:
        #         predictions.loc[x.index.intersection(node.samples.index)] = node.label
        #         break
        #     left_mask = left_samples.index.isin(x.index)
        #     right_mask = right_samples.index.isin(x.index)
        #     predictions.loc[left_mask] = node.left.predict(left_samples)
        #     predictions.loc[right_mask] = node.right.predict(right_samples)
        #     node = node.left if x.loc[left_mask, node.feature].all() else node.right
        
        # return predictions

        # создаем серию с предсказаниями и возвращаем ее
        # predictions = pd.Series(index=X.index)
        # for i, row in X.iterrows():
        #     predictions[i] = self._predict_row(row)
        # return predictions

        x = X.copy()
        node = self._node
        left_samples = x[x[node.feature] < node.threshold]
        right_samples = x[x[node.feature] >= node.threshold]
        x.loc[left_samples.index, "pred"] = self._left_node.dominative_class
        x.loc[right_samples.index,"pred"] = self._right_node.dominative_class
        return x.pred

    def _predict_row(self, row: pd.Series) -> int:
        # делаем предсказание для одной строки
        feature_value = row[self._node.feature_index]
        if feature_value < self._node.threshold:
            return self._left_node.prediction
        else:
            return self._right_node.prediction

    def fit_predict(self, X: pd.DataFrame, y: pd.Series) -> pd.Series:
        self.fit(X, y)
        return self.predict(X)


Проверим 

In [33]:
from sklearn.model_selection import train_test_split


stump = DecisionStump()
stump.fit(X_train, y_train)
y_pred = stump.predict(X_test)

print(classification_report(y_test, y_pred))
stump._node.feature, stump._node.threshold


reduced_df = df[df.target.apply(lambda t: t in ['setosa', 'versicolor'])]
X, y = reduced_df.drop(columns=['target']), reduced_df.target
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.35, random_state=42)

print(len(X_train), len(X_test))




              precision    recall  f1-score   support

      setosa       1.00      0.93      0.96        29
  versicolor       0.48      1.00      0.65        23
   virginica       0.00      0.00      0.00        23

    accuracy                           0.67        75
   macro avg       0.49      0.64      0.54        75
weighted avg       0.53      0.67      0.57        75

35 65


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:



from sklearn.metrics import classification_report

@dataclass
class DecisionNode:
    # Решающие признаки остаются как есть
    feature: Optional[str] = None
    feature_idx: Optional[int] = None
    threshold: Optional[float] = None
    # Добавляются ссылки на левую и правую вершины
    left: Optional['DecisionNode'] = None
    right: Optional['DecisionNode'] = None
    # Класс в листе остается как есть
    dominative_class: Any = None

    @property
    def is_leaf(self) -> bool:
        return bool(self.dominative_class)
class DecisionTree:
    def __init__(self, max_impurity_spllit: float = 0.01, min_leaf_split=2, max_depth=None):
        self._root: Optional[DecisionNode] = None
        # В качестве простых эвристики для остановки возьмем:
        # - максимальную глубину ветви
        # - минимально допустимое число примеров в узле
        # - долю "примесей" в узле
        self._max_impurity_spllit = max_impurity_spllit
        self._min_leaf_split = min_leaf_split
        self._max_depth = max_depth

    def _fit(self, X_train: pd.DataFrame, y_train: pd.Series, depth=0):
        # проверим, можно ли вернуть лист
        classes_probas = y_train.value_counts(normalize=True).to_dict()
        dominative_class = max(classes_probas, key=lambda c: classes_probas[c])
        impurity = 1 - classes_probas[dominative_class]
        leaf_size = len(X_train)
        if impurity <= self._max_impurity_spllit or leaf_size < self._min_leaf_split or (self._max_depth and depth >= self._max_depth):
            return DecisionNode(dominative_class=dominative_class)
        # если нет, продолжаем идти рекурсивно
        # делаем перебор максимум в 10 точках
        _, feature, feature_idx, threshold = split_by_features(X_train, y_train, 10)
        if not feature:  # если не получается выбрать оптимальный признак для разбиения - завершаем перебор
            return DecisionNode(dominative_class=dominative_class)
        x_left, x_right = X_train[X_train[feature] < threshold], X_train[X_train[feature] >= threshold]
        y_left, y_right = y_train[x_left.index], y_train[x_right.index]
        left_node = self._fit(x_left, y_left, depth + 1)
        right_node = self._fit(x_right, y_right, depth + 1)
        return DecisionNode(
            feature=feature,
            feature_idx=feature_idx,
            threshold=threshold,
            left=left_node,
            right=right_node,
        )

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        self._root = self._fit(X_train, y_train)

    def _predict(self, sample: pd.Series, node: DecisionNode) -> Any:
        if node.is_leaf:
            return node.dominative_class
        if sample[node.feature] < node.threshold:
            return self._predict(sample, node.left)
        return self._predict(sample, node.right)

    def predict(self, X: pd.DataFrame) -> pd.Series:
        return X.apply(lambda sample: self._predict(sample, self._root), axis=1)

    def fit_predict(self, X_train: pd.DataFrame, y_train: pd.Series):
        self.fit(X_train, y_train)
        return self.predict(X_train)




X, y = df.drop(columns=['target']), df.target
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5, random_state=42)

print(len(X_train), len(X_test))
tree = DecisionTree()
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)

print(classification_report(y_test, y_pred))



75 75


ValueError: ignored

Вторая часть ДЗ

В этой домашней работе вам предстоит научиться предсказывать цены товаров из маркетплейса Azamon.

Требования к домашней работе:
- Во всех графиках должны быть подписи через title, legend, etc.
- Во время обучения моделей проверяйте, что у вас не текут данные. Обычно это позитивно влияет на качество модели на тесте, но негативно влияет на оценку 🌚
- Если вы сдаете работу в Google Colaboratory, убедитесь, что ваша тетрадка доступна по ссылке.
- Использование мемов допускается, но необходимо соблюдать меру. Несодержательная работа, состоящая только из мемов, получает 0 баллов.

# Домашняя работа: деревья решений

Импорт модулей

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder


In [ ]:
df = pd.read_csv('amazon_co-ecommerce_sample.csv').drop(columns=[
    'product_name',
    'index',
    'uniq_id',
    'customers_who_bought_this_item_also_bought',
    'items_customers_buy_after_viewing_this_item',
    'sellers',
    'description', # text
    'product_information', # text
    'product_description', # text
    'customer_questions_and_answers', # text
    'customer_reviews', # text
])

ParserError: ignored

## Очистка данных (1 балл)

Посмотрите на признаки. Есть ли в них пропуски? Какое соотношение между NaN'ами и общим количеством данных? Есть ли смысл выкидывать какие-либо данные из этого датасета?

Считывание таблицы и удаление некоторых колонок

In [ ]:
def getna(df):
    print("Количество nan | имя топика | процент потери от общего числа")
    for column in df.columns:
        #print(df[column])
        print(df[column].isnull().sum(), "|", column, "|", df[column].isnull().sum() / len(df[column]) * 100, "%")

getna(df)

Вывод кол-ва NaN-ов, имени топика и соотношения между общим кол-вом данных и NaN-ов

Выкидывать данные из датасета не стоит, потому что все эти фичи крайне вероятно, что
 напрямую или косвенно будут влиять на будущую цену товара

 Заполнение пропусков в числовых признаках:


## Подготовка данных (3 балла)

Обработайте признаки. Выполните кодирование категориальных признаков, заполните пропуски в числовых признаках. Обратите внимание, что в датасете есть признак, который разбивается на несколько подпризнаков. Что это за признак? Закодируйте и его.

Дополнительные вопросы (+ 1 балл):
- Какие из признаков в этом датасете лучше кодировать через ordinal encoding?
- Какие из признаков допустимо кодировать через one-hot?

Прим.: суммарно за эту секцию можно получить до 4 баллов.

Колонка со средним пользовательским рейтингом переводится в систему оценки от 0 до 1

In [ ]:
formean = sum([float(x.split()[0]) / float(x.split()[-2]) for x in df["average_review_rating"] if type(x) == str])
print(formean / len(df)) # Mean value
df["average_review_rating"] = df["average_review_rating"].fillna(f"{formean * 5} out of 5 stars")
df["average_review_rating"] = [float(x.split()[0]) / float(x.split()[-2]) for x in df["average_review_rating"]]
print(df["average_review_rating"])

Колонка с кол-вом ответов на вопросы заполняется, где недостача, средними значениями

In [ ]:
df["number_of_answered_questions"] = df["number_of_answered_questions"].astype(float)
df["number_of_answered_questions"] = df["number_of_answered_questions"].fillna(df["number_of_answered_questions"].mean())
print(df["number_available_in_stock"])


Колонка с оценками заполняется 

In [ ]:
df["number_of_reviews"] = df["number_of_reviews"].fillna(df["number_of_answered_questions"]).astype(str)
df["number_of_reviews"] = [x.replace(",", ".") for x in df["number_of_reviews"]]
df["number_of_reviews"] = df["number_of_reviews"].astype(float)
print(df["number_of_reviews"])

Колонка кол-ва ревью преобразовывается в float тип и заполняется средними значениями

In [ ]:
df["number_of_reviews"] = df["number_of_reviews"].apply(lambda x: float(str(x).replace(",", '.')))
df["number_of_reviews"] = df["number_of_reviews"].fillna(df["number_of_reviews"].mean())
print(df["number_of_reviews"])

Колонка цен преобразовывается к float и заполняется средними значениями

In [ ]:
df.price = df.price.apply(lambda x: float(str(x).replace("£", "").replace(",", "").split()[0]))
df.price = df.price.fillna(df.price.mean())
print(df.price)

Колонка количества товара в наличии заполняется нулями, если товар отсутствует

In [ ]:
df["number_available_in_stock"] = df["number_available_in_stock"].apply(lambda x: float(str(x).split()[0]))
df["number_available_in_stock"] = df["number_available_in_stock"].fillna(0)
print(df["number_available_in_stock"])

Категориальные признаки

Колонка производителей с помощью Ordinal encoding кодируется

In [ ]:
X = df["manufacturer"].fillna(df["manufacturer"].value_counts().index[0])
ll = LabelEncoder()
X = ll.fit_transform(X)
df["manufacturer"] = X
print(df["manufacturer"])

Колонка с категориями амазон - колонка, которая представляет собой еще один датафрейм. 

In [ ]:
ll = LabelEncoder()
df["amazon_category_and_sub_category"] = df["amazon_category_and_sub_category"].fillna(df["amazon_category_and_sub_category"].mode().iloc[0])
df["amazon_category_and_sub_category"] = ll.fit_transform(df["amazon_category_and_sub_category"].values)
print(df["amazon_category_and_sub_category"])

## Бейзлайн

Обучите базовую модель. Для этого используйте `sklearn.dummy.DummyRegressor`. Какое качество она показывает на тесте? Посчитайте MSE, RMSE.

In [ ]:
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error, r2_score

DRegr = DummyRegressor()
X = df.drop(columns="price")
y = df.price

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)


Разделяем данные на обучающую выборку и тестовую

In [ ]:
DRegr.fit(X_train, y_train)
y_pred = DRegr.predict(X_test)
print(DRegr.score(X_test, y_test))

  Модель стала чуть хуже: Коэффициент детерминации прогноза, идеально 1, если меньше 0, то стало похуже


In [ ]:
MSE = mean_squared_error(y_test, y_pred, squared=True)
RMSE = mean_squared_error(y_test, y_pred, squared=False)
print(MSE, RMSE)

Вычисление значения функции ошибки с помощью встроенной в sklearn функции,
squared = True возвращает MSE, если False - RMSE

## Дерево решений

Обучите регрессионное дерево решений, проверьте качество этой модели на тестовой выборке. Улучшилось ли качество по сравнению с базовой моделью? Оцените r2_score обученной модели.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

DecTree = DecisionTreeRegressor()
DecTree.fit(X_train, y_train)

y_pred_dec = DecTree.predict(X_test)

print(r2_score(y_test, y_pred_dec))

## Линейная регрессия

Попробуйте обучить линейную регрессию с параметрами по умолчанию. Оцените r2_score на тестовой выборке. Сравните качество с деревом решений. 

In [ ]:
from sklearn.linear_model import LinearRegression

LinReg = LinearRegression()
LinReg.fit(X_train, y_train)

y_pred_lr = LinReg.predict(X_test)
print(r2_score(y_test, y_pred_lr))